# MLO Practical Exam - Nick Bischofberger

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from numpy import log, exp

from sklearn import preprocessing
from sklearn import impute
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, mean_squared_error, confusion_matrix
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn import decomposition as dcp
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

### Python navigation

In [ ]:
df[df["sex"]=="Male"]
df.loc[[5,8]] #df.loc[[observations],[columns]] OR df.loc[5:8,1:4] --> lists inside loc only if you want to access specific columns/observations that are not next to each other
df["Ticket"].loc[888]
df[df["size"]>=4]
df[df["Fare"]==df["Fare"].max()]
df.drop(columns=["time"],inplace=True)
df.drop(index=892, inplace=True)
df.groupby(["sex"]).mean()
df.groupby(["size","sex"]).count()
tips.groupby("day").mean()[["total_bill"]]
scientists.keys() #Dictionary

### Data pre-processing

#### Loading data and dropping columns

In [ ]:
df = pd.read_csv("Titanic.csv")
df.drop(columns=["time"],inplace=True)
df.drop(index=892, inplace=True)
df.corr()

#### Check for weird values

In [ ]:
df.info()
df.describe()
plt.hist(df['Carat Weight']) #plot histogram to see if values make sense
a =~ df["Age"].isna() #finds the locations where Age is *not* empty
plt.boxplot(df.loc[a,"Age"],vert=False) #filter based on the non-empty values, draw a horizontal boxplot
plt.boxplot(df["Fare"],vert=False)
plt.hist(df["Age"])
plt.scatter(Sarah_raw_data['Carat Weight'],Sarah_raw_data['Price'])

#### Feature encoding

In [ ]:
df["Pclass"].unique()
df["Name"].unique().shape

df = pd.get_dummies(df,columns=["Sex","Embarked"], drop_first=True) #Binary
df["Age"] = df.Age.map({"Young":0, "Mid-aged":1, "Old":2}) #Ordinary, df["target column"] = df."base_column"

#### Feautre Engineering

In [ ]:
df["Fare_euros_2021"] = df["Fare"]*1.18*118.36
df["Family_Presence"]=np.where((df["SibSp"]>=1) | (df["Parch"]>=1), 1,0)
df['Carat Weight:Color_E'] = df['Carat Weight'] * df['Color_E']

#### Missing values and imputing

In [ ]:
df.isna().sum()

imp = SimpleImputer(missing_values=np.nan, strategy="most_frequent") #other strategy: "mean"
df[["Embarked"]]=imp.fit_transform(df[["Embarked"]])
imp = KNNImputer(n_neighbors=1)
df[["Age"]]=imp.fit_transform(df[["Age"]])

#### Check for duplicates

In [ ]:
df.duplicated().sum() #how many duplicates in the dataset
df[df.duplicated()] #shows all duplicate rows. Add .count() at the end to see the # of duplicates per column
df.drop_duplicates(inplace=True)

#### Plotting

In [ ]:
plt.scatter(df["height"],df["weight"]) #plt.sth[x,y]
plt.xlabel("height")
plt.ylabel("weight")
plt.plot(X,y_pred,color="red")
plt.show()

#### Scaling data

In [ ]:
# Normalize between 0 and 1 (default)
min_max_scaler=preprocessing.MinMaxScaler()
X_minmax = min_max_scaler.fit_transform(X)
# Scaling (mean = 0, std = 1)
X_scaled = preprocessing.scale(X)

## Supervised Learning - Regression

#### X and Y split

In [ ]:
X = df.drop(columns="height")
y = df[["weight"]]

#### Train Test split

In [ ]:
X_train, X_other, y_train, y_other= train_test_split(X, y, test_size=0.5) #test_size is for second one
X_val, X_test, y_val, y_test= train_test_split(X_other, y_other, test_size=0.5)
X_final = pd.concat([X_train, X_val])
y_final = pd.concat([y_train, y_val])

### Linear Regression

In [ ]:
lm = LinearRegression().fit(X, y)
print("Intercept = ",lm.intercept_) # Print the resultant model intercept 
print("Model coefficients = ", lm.coef_) # Print the resultant model coefficients (in order of variables in X)
print("R^2 =",lm.score(X,y)) # Print the resultant model R-squared

y_pred=lm.predict(X)
plt.scatter(X,y)
plt.plot(X,y_pred,c="red")
plt.xlabel("height")
plt.ylabel("weight")

y_pred = lm.predict(X_val)
mean_squared_error(y_val,y_pred, squared=False) #RMSE

### Polynimoal Regression

In [ ]:
degree=3
poly = PolynomialFeatures(degree) #define the polynomial
X_poly=poly.fit_transform(X) #map all the values of X as [1,x,x^2,x^3, etc]
polyreg = LinearRegression().fit(X_poly, y)
print("Intercept = ",polyreg.intercept_) 
print("Model coefficients = ", polyreg.coef_)
print("R^2 =",polyreg.score(X_poly,y))

y_pred=polyreg.predict(X_poly) 

plt.scatter(X,y)
linepoints = np.linspace(np.min(X), np.max(X), 100) # create 100 points between 58 and 72
linepoints_poly=poly.fit_transform(linepoints) #transform these datapoints into polynomial datapoints
linepoints_pred=polyreg.predict(linepoints_poly) #then predict the value we would get on these points with our model
plt.plot(linepoints,linepoints_pred)

mean_squared_error(y_val,y_pred, squared=False) #RMSE

### Log-Log model

In [ ]:
X_train_log=X_train.copy()
X_val_log=X_val.copy()
y_train_log=y_train.copy()
X_train_log["Carat Weight"]=X_train_log["Carat Weight"].apply(log)
X_val_log["Carat Weight"]=X_val_log["Carat Weight"].apply(log)
y_train_log=y_train_log.apply(log)
lm = LinearRegression().fit(X_train_log, y_train_log)
print("Intercept = ",lm.intercept_) # Print the resultant model intercept 
print("Model coefficients = ", lm.coef_) # Print the resultant model coefficients (in order of variables in X)
print("R^2 =",lm.score(X,y)) # Print the resultant model R-squared
y_pred_log = lm.predict(X_val_log)
mean_squared_error(y_val,np.exp(y_pred_log),squared=False) #RMSE

## Supervised learning - Classification

### Logistic Regression

In [ ]:
logm = LogisticRegression(max_iter=2000).fit(X_train, y_train) # Fit a logistic regression with vector y as dependent and matrix X as independent
print("Intercept = ",logm.intercept_) # Print the resultant model intercept 
print("Model coefficients = ", logm.coef_) # Print the resultant model coefficients (in order of variables in X)
print("R^2 =",logm.score(X,y)) # Print the resultant model R-squared
logm.predict(X_val) #getting the predicted labels
y_pred_proba = logm.predict_proba(X_val)[:,1] #getting the predicted probabilities 

#### Find desired classification threshold

In [ ]:
#roccurve[0] = 1-specificity, roccurve[1] = sensitivity, roc curve [2] = evaluated thresholds
roccurve=metrics.roc_curve(y_val,y_pred_proba)
plt.plot(roccurve[0], roccurve[1], linewidth=4)
((roccurve[0]<=0.3) & (roccurve[0]>=0.2)) & (roccurve[1]>=0.9) #Pick your favorite threshold 
roccurve[1][((roccurve[0]<=0.3) & (roccurve[0]>=0.2)) & (roccurve[1]>=0.9)] #sensitivity
roccurve[2][((roccurve[0]<=0.3) & (roccurve[0]>=0.2)) & (roccurve[1]>=0.9)] #thresholds
roccurve[0][((roccurve[0]<=0.3) & (roccurve[0]>=0.2)) & (roccurve[1]>=0.9)] #1-specificity

In [ ]:
X_final = pd.concat([X_train,X_val])
y_final = pd.concat([y_train, y_val])

logm = LogisticRegression().fit(X_final, y_final)
y_pred_proba = logm.predict_proba(X_test)[:,1] #predictions of the probabilities on the test data
threshold = 0.018 #desired threshold
y_pred = np.where(y_pred_proba > threshold, 1, 0) # predicted label based on defined threshold

confusion_matrix(y_test,y_pred)
accuracy_score(y_test,y_pred)
roc_auc_score(y_test, y_pred_proba)

### Trees (Decision Tree, RandomForest and GradientBoosting)

In [ ]:
classifier_DT = DecisionTreeClassifier(max_leaf_nodes = 6)
# classifier_RF = RandomForestClassifier(max_leaf_nodes = 13)
# classifier_GBM = GradientBoostingClassifier(max_leaf_nodes = 13)
classifier_DT.fit(X_train, y_train) # change model to _RF or _GBM
y_pred_prob=classifier_DT.predict_proba(X_val)[:,1] # change model to _RF or _GBM
print(roc_auc_score(y_val, y_pred_prob))

#### Plot tree

In [ ]:
figure(figsize=(10,8)) #makes plot large
tree.plot_tree(classifier_DT,  class_names=("Died","Survived"), feature_names=X_train.columns, filled=True) # change model to _RF or _GBM

#### Find best number of leaves

In [ ]:
n_max_leaf_nodes = range(2,60) # Lets train the models with 2, 3, 4, ... 60 leafs
array_train = []
array_val= []
for n in n_max_leaf_nodes:
    classifier_DT = DecisionTreeClassifier(max_leaf_nodes = n).fit(X_train, y_train)
    #classifier_DT = RandomForestClassifier(max_leaf_nodes = n).fit(X_train, y_train)
    #classifier_DT = GradientBoostingClassifier(max_leaf_nodes = n).fit(X_train, y_train)
    y_pred_train = classifier_DT.predict_proba(X_train)[:,1] # change model to _RF or _GBM
    y_pred_val = classifier_DT.predict_proba(X_val)[:,1] # change model to _RF or _GBM
    score_train=roc_auc_score(y_train,y_pred_train)
    score_val=roc_auc_score(y_val,y_pred_val)
    array_train.append(score_train)
    array_val.append(score_val)

plt.scatter(n_max_leaf_nodes,array_train)
plt.scatter(n_max_leaf_nodes,array_val)
plt.legend(['Training set','Validation set'])
plt.xlabel("Number of leaves",fontsize=15)
plt.ylabel("AUC",fontsize=15)

#### Final model

In [ ]:
classifier_DT = DecisionTreeClassifier(max_leaf_nodes = 10) #optimal # of leaf nodes
#classifier_RF = RandomForestClassifier(max_leaf_nodes = 15) #optimal # of leaf nodes
#classifier_GBM = GradientBoostingClassifier(max_leaf_nodes = 15) #optimal # of leaf nodes

X_final = pd.concat([X_train,X_val])
y_final = pd.concat([y_train, y_val])

classifier_DT.fit(X_final, y_final) # change model to _RF or _GBM
y_pred = classifier_DT.predict(X_test) # change model to _RF or _GBM
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

#### Change the classification threshold

In [ ]:
classifier_DT = DecisionTreeClassifier(max_leaf_nodes = 10) #optimal # of leaf nodes
#classifier_RF = RandomForestClassifier(max_leaf_nodes = 15) #optimal # of leaf nodes
#classifier_GBM = GradientBoostingClassifier(max_leaf_nodes = 15) #optimal # of leaf nodes
classifier_DT.fit(X_final, y_final)
y_pred_prob=classifier_DT.predict_proba(X_test)[:,1] #different from 
threshold=0.7
y_pred=np.where(y_pred_prob>threshold,1,0)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

#### Feature importance

In [ ]:
feat_importances = pd.Series(classifier_DT.feature_importances_, index=X_final.columns) # change model to _RF or _GBM
feat_importances.nlargest(5).plot(kind='barh') #select # features to show

## Unsupervised Learning

### Principal Component Analysis (PCA)

In [ ]:
df.cov()
df["Age"].var()/df.var().sum() # divide variance of every column by total variance to get relative variance

df=pd.DataFrame()
#creating centered variables
Vit_C_centered=menu["Vitamin C"]-menu["Vitamin C"].mean()
Total_Fat_centered=menu["Total Fat"]-menu["Total Fat"].mean()
Cholesterol_centered=menu["Cholesterol"]-menu["Cholesterol"].mean()
df["z1"]=-0.17718634*Vit_C_centered+0.54454878*Total_Fat_centered+0.81979975*Cholesterol_centered
df["z2"]=0.98405437*Vit_C_centered+0.08485918*Total_Fat_centered+0.15631992*Cholesterol_centered
df["z3"]=0.01555629*Vit_C_centered+0.83442528*Total_Fat_centered-0.5509149*Cholesterol_centered

pca=dcp.PCA(n_components=3) # change to desired number of principal components
pca.fit(df)
loadings = pca.components_ #loadings
pca.explained_variance_ #variance
rel_var = pca.explained_variance_ratio_ #relative variance
data_pca = pca.fit_transform(df) #scores
pd.DataFrame(data=[loadings[0], loadings[1]], index=["z1","z2"],columns=menu_num.columns) #adjust loadings [0] and z1,z2,... to number of principal components
cumsum = np.cumsum(rel_var)
plt.title("Explained Variance Ratio by Component")
plt.plot(range(1,11),cumsum) # change to desired number of principal components
plt.xlabel("Component")
plt.ylabel("Variance Ratio")
plt.show()
index = np.where(cumsum>=0.95) # first number in array that is returned is the number of prinipal components to keep to get 95% explained variance

### KMeans Clustering

In [ ]:
df = preprocessing.scale(df) # scale before KMeans
kmeans = KMeans(n_clusters=4).fit(df) #specify number of desired clusters
labels = kmeans.labels_ #clusters, clusters start from 0
df["Cluster"] = labels # add labels to dataframe
kmeans.cluster_centers_ #optimized centers = centroids
print(df[df["Cluster"]==1].shape) #How many are in cluster 1
df[df["Cluster"]==1].mean().sort_values().tail() #Gives you the mean for each column in cluster 1

#### Find optimal number of clusters

In [ ]:
#Question, this is to select the right K, right?
inertia_K=[] #sum of all distances from centroid to points squared, we want low inertia
K = range(1,10) #select range of clusters you want
for k in K:
    kmeans =KMeans(n_clusters=k).fit(df)
    inertia_K.append(kmeans.inertia_)
plt.plot(K,inertia_K) #Question: Can we not replace range with K?

### Hierarchial Clustering

In [ ]:
df = preprocessing.scale(df) # scale before Hierarch. Clustering
Z = linkage(df,method='average') #other methods: "average","single", "complete", "ward"
dendrogram(Z)

In [ ]:
labels = fcluster(Z, 3, criterion='maxclust') #The # is the number of clusters?
df["Cluster"] = labels #add labels to dataframe
print(df[df["Cluster"]==1].shape) #How many are in cluster 1
df[df["Cluster"]==1].mean().sort_values().tail() #Gives you the mean for each column in cluster 1

#### Visualize average scores for cluster

In [ ]:
# KMeans
df_pca_km= df_pca.copy() #df_pca = scores
df_pca_km["Labels"] = labels #use the labels obtained
df_pca_km_clust=df_pca_km.groupby(df_pca_km["Labels"]).mean().reset_index() #take the average over the 5 new features based on the clusters
df_pca_km_clust=df_pca_km_clust.drop(columns=["Labels"]).set_index(np.arange(1,6)) #obtain a dataframe that contains this information

sns.heatmap(df_pca_km_clust,cmap="PiYG") #plot a heatmap of this
plt.xlabel('Average score for each feature', fontsize = 15) # x-axis label with fontsize 15
plt.ylabel('Cluster', fontsize = 15) # y-axis label with fontsize 15

In [ ]:
# Hierarchical clustering
df_pca_hier = df_pca.copy()
df_pca_hier["Labels"]=labels # hierarchical_clustering
df_pca_hier_clust = df_pca_hier.groupby(df_pca_hier["Labels"]).mean().reset_index()
df_pca_hier_clust=df_pca_hier_clust.drop(columns=["Labels"]).set_index(np.arange(1,6))

sns.heatmap(df_pca_hier_clust,cmap="PiYG")
plt.xlabel('Average score for each feature', fontsize = 15) # x-axis label with fontsize 15
plt.ylabel('Cluster', fontsize = 15) # y-axis label with fontsize 15